In [1]:
import pandas as pd
import numpy as np
import torch
import random
import os
import sys

# ==========================================
# 1. 라이브러리 및 환경 설정
# ==========================================
try:
    from sklearn.metrics import classification_report, accuracy_score, f1_score
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, BigBirdConfig
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
except ImportError:
    sys.exit("❌ 라이브러리가 설치되지 않았습니다. !pip install transformers datasets accelerate scikit-learn 실행 필요")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 [KoBigBird] 사용 장치: {device}")

# ==========================================
# 2. 데이터 로드 및 병합
# ==========================================
url = "https://raw.githubusercontent.com/tunib-ai/DKTC/main/data/train.csv"
try:
    df_threat = pd.read_csv(url)[['class', 'conversation']]
    print(f"✅ 위협 데이터 로드 완료: {len(df_threat)}개")
except Exception as e:
    sys.exit(f"❌ 위협 데이터 로드 실패: {e}")

# 일반 대화 파일 로드 (파일명 확인)
normal_file = "normal_conversation (1).csv"
if not os.path.exists(normal_file):
    if os.path.exists("normal_conversation.csv"):
        normal_file = "normal_conversation.csv"
    else:
        sys.exit(f"❌ '{normal_file}' 파일을 찾을 수 없습니다.")

print(f"📂 사용할 일반 대화 파일: {normal_file}")
df_normal = pd.read_csv(normal_file)
df_normal['conversation'] = df_normal['conversation'].str.replace(r'(^|\n)[AB]:\s*', '', regex=True)
if 'class' not in df_normal.columns:
    df_normal['class'] = '일반 대화'
df_normal = df_normal[['class', 'conversation']]
print(f"✅ 일반 대화 데이터 로드 완료: {len(df_normal)}개")

# 병합
df_final = pd.concat([df_threat, df_normal], ignore_index=True)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

# ==========================================
# 3. 라벨 인코딩 & 데이터셋 변환
# ==========================================
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df_final['label'] = df_final['class'].map(label_map)
df_final = df_final.dropna(subset=['label'])
df_final['label'] = df_final['label'].astype(int)

train_df, val_df = train_test_split(df_final, test_size=0.2, random_state=42, stratify=df_final['label'])
train_dataset = Dataset.from_pandas(train_df[['conversation', 'label']])
val_dataset = Dataset.from_pandas(val_df[['conversation', 'label']])

# ==========================================
# 4. 모델 & 토크나이저 (KoBigBird)
# ==========================================
# 🚨 [변경됨] KoBigBird 모델 사용
MODEL_NAME = "monologg/kobigbird-bert-base"
print(f"🔄 모델 로딩 중: {MODEL_NAME}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 🚨 [중요] BigBird 설정: 짧은 문장(4096 토큰 미만) 처리 시 original_full 사용 권장
# 만약 메모리가 부족하면 이 부분을 삭제하여 기본값(block_sparse)으로 사용하세요.
config = BigBirdConfig.from_pretrained(MODEL_NAME)
config.attention_type = "original_full"
config.num_labels = 5

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config).to(device)

def preprocess_function(examples):
    # BigBird는 긴 문맥(최대 4096) 처리가 장점이지만,
    # 이번 데이터셋은 대화가 그렇게 길지 않으므로 512~1024 정도로 설정해도 충분합니다.
    # 여기서는 안전하게 512로 설정합니다. (필요 시 1024로 늘리세요)
    return tokenizer(examples["conversation"], truncation=True, max_length=512)

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==========================================
# 5. 학습 (Training)
# ==========================================
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="./results_bigbird",  # [변경됨] 저장 폴더
    learning_rate=2e-5,
    per_device_train_batch_size=8,   # [변경됨] 메모리 절약을 위해 8로 조정 (가능하면 16)
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,   # 배치 8 * 2 = 실제 16 효과
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\n🦅 KoBigBird 학습 시작! (시간이 조금 더 걸릴 수 있습니다)")
trainer.train()

# ==========================================
# 6. 모델 저장
# ==========================================
SAVE_PATH = "./final_model_bigbird" # [변경됨] 저장 폴더
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print(f"\n✅ 학습 완료! 모델이 '{SAVE_PATH}' 폴더에 저장되었습니다.")

🚀 [KoBigBird] 사용 장치: cuda
✅ 위협 데이터 로드 완료: 3950개
📂 사용할 일반 대화 파일: normal_conversation (1).csv
✅ 일반 대화 데이터 로드 완료: 800개
🔄 모델 로딩 중: monologg/kobigbird-bert-base


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


✅ 학습 완료! 모델이 './final_model_bigbird' 폴더에 저장되었습니다.


In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================================
# 1. 설정 및 데이터 준비
# ==========================================
MODEL_PATH = "./final_model_bigbird"       # 학습된 모델 경로
DATA_PATH = "final_train_dataset.csv" # 데이터 파일 경로

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 로드
df = pd.read_csv(DATA_PATH)
df['conversation'] = df['conversation'].astype(str)

# 라벨 재매핑 (안전장치)
label_map = {
    '협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3,
    '협박': 0, '갈취': 1, '직장 내 괴롭힘': 2, '기타 괴롭힘': 3,
    '직장 괴롭힘': 2, '기타 괴롭힘': 3,
    '일반 대화': 4
}
df['label'] = df['class'].map(label_map)

# 결측치 제거
df = df.dropna(subset=['label'])
df['label'] = df['label'].astype(int)

# ---------------------------------------------------------
# ⚖️ [핵심] 클래스별 100개씩 균형 샘플링
# ---------------------------------------------------------
# 각 라벨별로 랜덤하게 100개씩 뽑습니다.
try:
    test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)
    print(f"테스트 데이터셋 구성 완료: 총 {len(test_df)}개")
    print(test_df['class'].value_counts()) # 각 100개인지 확인
except ValueError as e:
    print(f"오류: 데이터가 부족하여 클래스별 100개를 뽑을 수 없습니다. ({e})")
    # 예외 시 전체 데이터 사용
    test_df = df

# ==========================================
# 2. 모델 로드 및 예측
# ==========================================
print("\n모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (코드가 멈추지 않도록 임시 종료 처리 필요 시 exit())

# 예측 함수
def predict_batch(texts, batch_size=32):
    all_preds = []
    # 데이터가 많을 경우 배치를 나눠서 처리
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        all_preds.extend(preds)
    return np.array(all_preds)

print("예측 시작... (잠시만 기다려주세요)")
y_true = test_df['label'].tolist()
y_pred = predict_batch(test_df['conversation'].tolist())

# ==========================================
# 3. 평가 지표 출력 (Text Only)
# ==========================================
target_names = ['협박', '갈취', '직장 괴롭힘', '기타 괴롭힘', '일반 대화']

print("\n" + "="*50)
print("[최종 모델 평가 리포트 (Test Set: 100 samples/class)]")
print("="*50)

# 1. Accuracy & F1-Score
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted') # 각 클래스 비중이 같으므로 macro와 유사함

print(f"정확도 (Accuracy): {acc:.4f}")
print(f"F1 점수 (Weighted): {f1:.4f}")
print("-" * 50)

# 2. Classification Report (상세 지표)
print("\n[클래스별 상세 지표]")
# digits=4 옵션으로 소수점 4자리까지 정밀하게 출력
print(classification_report(y_true, y_pred, target_names=target_names, digits=4))
print("="*50)

/tmp/ipython-input-2113530880.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = df.groupby('label').apply(lambda x: x.sample(n=100, random_state=42)).reset_index(drop=True)


테스트 데이터셋 구성 완료: 총 500개
class
협박 대화          100
갈취 대화          100
직장 내 괴롭힘 대화    100
기타 괴롭힘 대화      100
일반 대화          100
Name: count, dtype: int64

모델 로딩 중...


Loading weights:   0%|          | 0/203 [00:00<?, ?it/s]

예측 시작... (잠시만 기다려주세요)

[최종 모델 평가 리포트 (Test Set: 100 samples/class)]
정확도 (Accuracy): 0.9580
F1 점수 (Weighted): 0.9577
--------------------------------------------------

[클래스별 상세 지표]
              precision    recall  f1-score   support

          협박     0.9151    0.9700    0.9417       100
          갈취     0.9574    0.9000    0.9278       100
      직장 괴롭힘     0.9615    1.0000    0.9804       100
      기타 괴롭힘     0.9684    0.9200    0.9436       100
       일반 대화     0.9901    1.0000    0.9950       100

    accuracy                         0.9580       500
   macro avg     0.9585    0.9580    0.9577       500
weighted avg     0.9585    0.9580    0.9577       500



In [5]:
import pandas as pd
import json
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm


# 1. 설정 및 데이터 로드
MODEL_PATH = "./final_model_bigbird"       # 학습된 모델 경로
TEST_FILE = "/content/test (1).json"            # 업로드된 테스트 파일

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

# 테스트 데이터 로드 (JSON)
try:
    with open(TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)
    print(f"테스트 데이터 로드 완료: {len(test_data)}개")

    # 데이터 구조 확인 (첫 번째 아이템)
    first_key = list(test_data.keys())[0]
    print(f"데이터 예시: ID={first_key}, 내용={test_data[first_key]}")

except FileNotFoundError:
    print("오류: 'test.json' 파일을 찾을 수 없습니다.")
    # (테스트용 더미 데이터)
    test_data = {"t_000": {"text": "이거 들어봐 와 이 노래 진짜 좋다"}, "t_001": {"text": "야 돈 내놔"}}


# 2. 모델 로드
print("모델 로딩 중...")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH).to(device)
    model.eval()
except OSError:
    print("오류: 저장된 모델을 찾을 수 없습니다. 학습 코드를 먼저 실행했는지 확인해주세요.")
    # (임시 모델 로드)
    tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
    model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5).to(device)

# 3. 추론 (Inference)
print("예측 시작...")

results = []

# test_data가 딕셔너리 형태 {"t_000": {"text": "..."}} 라고 가정
for idx, item in tqdm(test_data.items()):
    # 텍스트 추출 (text 키가 없으면 conversation 등 다른 키 시도)
    text = item.get('text', item.get('conversation', ''))

    # 전처리 및 토크나이징
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 예측값 (0~4)
    pred_label = torch.argmax(logits, dim=-1).item()

    # 결과 저장
    results.append({
        'idx': idx,
        'class': pred_label  # 숫자로 저장 (0, 1, 2, 3, 4)
    })

# 4. 제출 파일 생성
submission = pd.DataFrame(results)

# 컬럼 순서 정렬 (idx, class)
submission = submission[['idx', 'class']]

# 파일 저장
save_path = "submission.csv"
submission.to_csv(save_path, index=False)

print("\n" + "="*50)
print(f"제출 파일 생성 완료: {save_path}")
print("="*50)
print(submission.head())

사용 장치: cuda
테스트 데이터 로드 완료: 500개
데이터 예시: ID=t_000, 내용={'text': '아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나보네 그럼 취소할까요 아가씨 내 여기단골이니 담에 갖다줄께 저도 알바생이라 외상안됩니다 아따 누가 떼먹는다고 그러나 갖다준다고 안됩니다 자꾸이럼 경찰불러요 아가씨 담배피는교 그건 왜 물으세요 그람 아가씨 담배 한대만 빌립시다 내 지금 지갑도 잃어버리고 기분이 그래서 그러니 여기요  아따 주는김에 한개더 주면 되겠네'}
모델 로딩 중...


Loading weights:   0%|          | 0/203 [00:00<?, ?it/s]

예측 시작...


100%|██████████| 500/500 [00:06<00:00, 75.13it/s]


제출 파일 생성 완료: submission.csv
     idx  class
0  t_000      1
1  t_001      2
2  t_002      2
3  t_003      3
4  t_004      3


In [3]:
import os
import pandas as pd

# ==========================================
# 2. 데이터 로드 및 병합 (전처리 포함)
# ==========================================

# 1) 기존 GitHub 위협 데이터 로드
url = "https://raw.githubusercontent.com/tunib-ai/DKTC/main/data/train.csv"
try:
    df_threat = pd.read_csv(url)[['class', 'conversation']]
    print(f"✅ 위협 데이터 로드 완료: {len(df_threat)}개")
except Exception as e:
    print(f"❌ 위협 데이터 로드 실패: {e}")
    df_threat = pd.DataFrame(columns=['class', 'conversation'])

# 2) 기존 일반 대화 파일 로드 및 전처리
normal_file = "normal_conversation (1).csv"
if not os.path.exists(normal_file):
    if os.path.exists("normal_conversation.csv"):
        normal_file = "normal_conversation.csv"

if os.path.exists(normal_file):
    df_normal = pd.read_csv(normal_file)
    # A:, B: 태그 제거
    df_normal['conversation'] = df_normal['conversation'].str.replace(r'(^|\n)[AB]:\s*', '', regex=True)
    if 'class' not in df_normal.columns:
        df_normal['class'] = '일반 대화'
    df_normal = df_normal[['class', 'conversation']]
    print(f"✅ 일반 대화 데이터 로드 완료: {len(df_normal)}개")
else:
    df_normal = pd.DataFrame(columns=['class', 'conversation'])

# 3) 새로 업로드한 CSV 데이터 로드 및 전처리 (가해자/피해자 제거)
new_data_file = "new_data_695.csv"
if os.path.exists(new_data_file):
    df_new = pd.read_csv(new_data_file)
    
    # 헤더 중복 포함된 경우 제거
    df_new = df_new[df_new['class'] != 'class'].reset_index(drop=True)
    
    # '가해자:', '피해자:' 태그 제거 (정규표현식 사용)
    df_new['conversation'] = df_new['conversation'].str.replace(r'(가해자|피해자)\s*:\s*', '', regex=True)
    # 양끝 불필요한 공백 제거
    df_new['conversation'] = df_new['conversation'].str.strip()
    
    df_new = df_new[['class', 'conversation']]
    print(f"✅ 새 데이터 로드 및 전처리 완료: {len(df_new)}개")
else:
    print(f"❌ '{new_data_file}' 파일을 찾을 수 없습니다. 파일명을 확인해 주세요.")
    df_new = pd.DataFrame(columns=['class', 'conversation'])

# 4) 모든 데이터 병합
df_final = pd.concat([df_threat, df_normal, df_new], ignore_index=True)

# 데이터 셔플
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"\n📊 최종 병합 데이터 수: {len(df_final)}개")
print(df_final['class'].value_counts())

✅ 위협 데이터 로드 완료: 3950개
✅ 일반 대화 데이터 로드 완료: 800개
✅ 새 데이터 로드 및 전처리 완료: 692개

📊 최종 병합 데이터 수: 5442개
class
갈취 대화          1152
기타 괴롭힘 대화      1094
협박 대화          1066
직장 내 괴롭힘 대화     979
일반 대화           800
직장 내 괴롭힘        177
기타 괴롭힘          174
Name: count, dtype: int64


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BigBirdConfig

# 1. 장치 설정 (GPU 사용 가능 여부 확인)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 사용 장치: {device}")

# 2. 모델 이름 설정
MODEL_NAME = "monologg/kobigbird-bert-base"

# 3. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 4. 모델 설정 및 로드 (클래스 5개)
config = BigBirdConfig.from_pretrained(MODEL_NAME)
config.attention_type = "original_full" 
config.num_labels = 5

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, config=config).to(device)

print("✅ 모델 및 토크나이저 로드 완료!")

🚀 사용 장치: cuda


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BigBirdForSequenceClassification LOAD REPORT from: monologg/kobigbird-bert-base
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
classifier.out_proj.weight                 | MISSING    | 
classifier.out_proj.bias                   | MISSING    | 
classifier.dense.weight                    | MISSING    | 
classifier.dense.bias                      | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if 

✅ 모델 및 토크나이저 로드 완료!


In [8]:
# 토큰화 함수 정의
def preprocess_function(examples):
    return tokenizer(examples["conversation"], truncation=True, padding=True, max_length=512)

# 준비된 데이터셋에 적용
tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)

print("✅ 토큰화가 완료되었습니다!")

Map:   0%|          | 0/4353 [00:00<?, ? examples/s]

Map:   0%|          | 0/1089 [00:00<?, ? examples/s]

✅ 토큰화가 완료되었습니다!


In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# 평가 지표 함수
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    return {"accuracy": acc, "f1": f1}

# 데이터 콜레이터 (패딩 처리)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results_bigbird_final",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=50,
    report_to="none"
)

# 트레이너 정의
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

print("\n🚀 5개 클래스 통합 학습을 시작합니다!")
trainer.train()


🚀 5개 클래스 통합 학습을 시작합니다!


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.670905,0.291518,0.910009,0.909384
2,0.520882,0.306269,0.919192,0.918359
3,0.223896,0.305576,0.927456,0.927348
4,0.207888,0.330718,0.930211,0.929960
5,0.132112,0.326637,0.930211,0.930183


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

TrainOutput(global_step=1365, training_loss=0.4652526052006872, metrics={'train_runtime': 2270.2945, 'train_samples_per_second': 9.587, 'train_steps_per_second': 0.601, 'total_flos': 5203780757665014.0, 'train_loss': 0.4652526052006872, 'epoch': 5.0})

In [10]:
# 저장 경로 설정
SAVE_PATH = "./final_model_kobigbird_v1"

# 모델과 토크나이저 저장
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print(f"✅ 모델이 '{SAVE_PATH}'에 안전하게 저장되었습니다!")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 모델이 './final_model_kobigbird_v1'에 안전하게 저장되었습니다!


In [11]:
from sklearn.metrics import classification_report

# 검증 데이터에 대한 예측값 얻기
predictions = trainer.predict(tokenized_val)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# 상세 리포트 출력
target_names = ['협박', '갈취', '직장 괴롭힘', '기타 괴롭힘', '일반 대화']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

          협박       0.85      0.93      0.89       213
          갈취       0.88      0.92      0.90       231
      직장 괴롭힘       0.94      0.95      0.94       231
      기타 괴롭힘       0.92      0.80      0.85       254
       일반 대화       1.00      1.00      1.00       160

    accuracy                           0.91      1089
   macro avg       0.92      0.92      0.92      1089
weighted avg       0.91      0.91      0.91      1089



In [12]:
import json
import pandas as pd
import torch
from tqdm import tqdm

# 1. 설정
TEST_FILE = "test.json"  # 파일 경로가 다르면 수정해 주세요 (예: "/content/test.json")
SAVE_PATH = "submission.csv"

# 2. 테스트 데이터 로드
try:
    with open(TEST_FILE, 'r', encoding='utf-8') as f:
        test_data = json.load(f)
    print(f"✅ 테스트 데이터 로드 완료: {len(test_data)}개")
except FileNotFoundError:
    print(f"❌ '{TEST_FILE}' 파일을 찾을 수 없습니다. 경로를 확인해 주세요.")

# 3. 추론 (Inference)
model.eval()
results = []

print("🚀 예측 시작...")
for idx, item in tqdm(test_data.items()):
    # 텍스트 추출 (text 또는 conversation 키 확인)
    text = item.get('text', item.get('conversation', ''))
    
    # 토큰화 및 장치 이동
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        padding=True, 
        max_length=512
    ).to(device)

    # 예측
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_label = torch.argmax(logits, dim=-1).item()

    # 결과 저장 (캐글 형식: idx, class)
    results.append({
        'idx': idx,
        'class': pred_label
    })

# 4. CSV 저장
submission = pd.DataFrame(results)
submission.to_csv(SAVE_PATH, index=False)

print("\n" + "="*50)
print(f"✅ 제출 파일 생성 완료: {SAVE_PATH}")
print("="*50)
print(submission.head())

✅ 테스트 데이터 로드 완료: 500개
🚀 예측 시작...


100%|██████████| 500/500 [00:06<00:00, 74.24it/s]


✅ 제출 파일 생성 완료: submission.csv
     idx  class
0  t_000      1
1  t_001      2
2  t_002      2
3  t_003      3
4  t_004      3
